In [1]:
# Decoding script, takes a GroupData set and uses Linear Discriminant Analysis
# to decode trial conditions or word tokens from neural data

import numpy as np
import os
import pickle
import ieeg.calc.stats
from matplotlib import pyplot as plt

# import matplotlib
# matplotlib.use('notebook')
import sys; print('Python %s on %s' % (sys.version, sys.platform))
sys.path.extend(os.path.expanduser(os.path.join('~', 'Documents', 'git', 'SentenceRep_analysis')))
sys.path.extend(os.path.expanduser(os.path.join('~', 'Documents', 'git', 'IEEG_Pipelines')))
from analysis.grouping import GroupData
from analysis.decoding import Decoder, get_scores, plot_all_scores, plot_dist_bound
from analysis.utils.plotting import plot_horizontal_bars
from analysis.decoding.words import score, dict_to_structured_array
box = os.path.expanduser(os.path.expanduser(os.path.join("~", "Box")))
fpath = os.path.join(box, "CoganLab")
subjects_dir = os.path.join(box, "ECoG_Recon")

Python 3.12.5 | packaged by conda-forge | (main, Aug  8 2024, 18:24:51) [MSC v.1940 64 bit (AMD64)] on win32






In [4]:
# %% Imports for total GroupData

sub = GroupData.from_intermediates(
    "SentenceRep", fpath, folder='stats', subjects_dir=subjects_dir)
all_data = []
colors = ['red', 'orange', 'yellow', 'blue']
#colors = ['paleturquoise', 'darkslategray', 'midnightblue', 'cornflowerblue']
scores = {'Hipp': None, 'Hipp&Sig': None, 'MaxHipp&Sig': None, 'MaxHipp': None}
scores2 = {'Hipp': None, 'Hipp&Sig': None, 'MaxHipp&Sig': None, 'MaxHipp': None}
idxs = [sub.AUD, sub.SM, sub.PROD, sub.sig_chans]
idxs = [list(idx) for idx in idxs]
names = list(scores.keys())
conds = [['aud_ls', 'aud_lm'], ['go_ls', 'go_lm'], 'resp']
window_kwargs = {'window': 20, 'obs_axs': 1, 'normalize': 'true', 'n_jobs': 30,
                'average_repetitions': False}

Loading power:   6%|▌         | 2/34 [00:00<00:11,  2.67it/s]


KeyboardInterrupt: 

In [14]:
conds = ['aud_ls', 'go_ls', 'resp']
reduced = sub[:, conds][:, :, :, idxs_maxhipp]
reduced.array = reduced.array.dropna()
reduced = reduced.nan_common_denom(True, 5, False)
comb = reduced.combine(('epoch', 'trial'))['zscore']
comb = comb.array.dropna().combine((0, 2))

In [26]:
# test stuff
X = comb[:, 'aud_ls']
print(X.labels)
from analysis.decoding import classes_from_labels
cats, labels = classes_from_labels(X.labels[1], crop=slice(0,4))
print(cats)
print(labels)
cats = {k: i for i, k in cats.items()}
stim_labels = [cats[l] for l in labels]
new_dict = {'heat': 0, 'hoot': 1, 'hot': 1, 'hut': 2}
decoder_labels = np.array([new_dict[s] for s in stim_labels])
print(decoder_labels)

[['D0008-LB1', 'D0008-LB3', 'D0008-LC1', 'D0008-LC2', 'D0008-RA1', 'D0008-RA2', 'D0008-RB1', 'D0008-RB2', 'D0008-RB3', 'D0008-RC1', 'D0008-RC2', 'D0008-RC3', 'D0008-RG1', 'D0008-RG2', 'D0009-LAMT1', 'D0009-LAMT2', 'D0009-LAMT3', 'D0009-LMST1', 'D0009-LMST2', 'D0009-RAMT1', 'D0009-RAMT2', 'D0009-RMST1', 'D0009-RMST2', 'D0009-RMST3', 'D0015-LMST1', 'D0015-LMST2', 'D0015-LMST3', 'D0015-LMST4', 'D0015-LPST1', 'D0015-LPST2', 'D0015-LPST3', 'D0015-LPST4', 'D0015-LPST5', 'D0015-LAMT1', 'D0015-LPIT2', 'D0018-LAST1', 'D0018-LAST2', 'D0018-LAST3', 'D0018-LAST4', 'D0018-LPMT1', 'D0018-LPMT2', 'D0018-LPMT3', 'D0018-LPMT4', 'D0018-RAST1', 'D0018-RAST2', 'D0018-RPMT1', 'D0018-RPMT2', 'D0018-RPMT3', 'D0022-LMIF1', 'D0022-LMIF2', 'D0027-LMMT1', 'D0027-LMMT2', 'D0027-LMMT3', 'D0027-LMMT4', 'D0027-LPMT1', 'D0027-LPMT2', 'D0027-LPMT3', 'D0027-LPMT4', 'D0027-LPMT5', 'D0027-LMST1', 'D0028-LPMT4', 'D0053-RPMT5', 'D0053-RPMT6', 'D0053-RPMT7', 'D0053-RPMT8', 'D0059-LASP3', 'D0059-LASP4', 'D0059-LAT1', 'D0059-

In [ ]:
from analysis.check.chan_utils import get_ch_label, get_preferred_ch_label, sep_sub_ch, regen_ch_label

In [ ]:
# To regenerate channel label and save as channel_label.pkl
sub_channels = sub.keys.get('channel',[])
orig_ch_label = get_ch_label(sub_channels, subjects_dir) 
maxhipp_ch_label = get_preferred_ch_label(sub_channels, subjects_dir, "Hipp", 0.1) # at least 10% 10mm-radius sphere overlap with hippocampus
with open('sub_channel.pkl', 'wb') as f:
    pickle.dump(sub_channels, f)
with open('channel_label.pkl', 'wb') as f:
    pickle.dump(orig_ch_label, f)
with open('maxhipp_channel_label.pkl', 'wb') as f:
    pickle.dump(maxhipp_ch_label, f)

#ch_roi = regen_ch_label('D0102-RTPI5', subjects_dir)
#print(sub_channels.find('D0102-RTPI5'))
#sub_channels = sub.keys.get('channel',[])

In [7]:
with open('sub_channel.pkl', 'rb') as f:
    sub_channels = pickle.load(f)
with open('channel_label.pkl', 'rb') as f:
    orig_ch_label = pickle.load(f)
with open('maxhipp_channel_label.pkl', 'rb') as f:
    maxhipp_ch_label = pickle.load(f)

idxs_hipp = [index for index, item in enumerate(orig_ch_label) if 'Hipp' in item]
idxs_hippsig = list(set(sub.sig_chans) & set(idxs_hipp))
idxs_maxhipp = [index for index, item in enumerate(maxhipp_ch_label) if 'Hipp' in item]
idxs_maxhippsig = list(set(sub.sig_chans) & set(idxs_maxhipp))

idxs_thal = [index for index, item in enumerate(orig_ch_label) if 'tha' in item]


In [ ]:
# Chi sq test for L and R hemi significant channel difference - not significant 
# selected_ch = [maxhipp_ch_label[i] for i in idxs_maxhipp]
# LHmaxhipp_count = sum("_L" in item for item in selected_ch)
# RHmaxhipp_count = sum("_R" in item for item in selected_ch)
# selected_ch = [maxhipp_ch_label[i] for i in idxs_maxhippsig]
# LHmaxhippsig_count = sum("_L" in item for item in selected_ch)
# RHmaxhippsig_count = sum("_R" in item for item in selected_ch)
# selected_ch = [orig_ch_label[i] for i in idxs_hipp]
# LHhipp_count = sum("_L" in item for item in selected_ch)
# RHhipp_count = sum("_R" in item for item in selected_ch)
# selected_ch = [orig_ch_label[i] for i in idxs_hippsig]
# LHhippsig_count = sum("_L" in item for item in selected_ch)
# RHhippsig_count = sum("_R" in item for item in selected_ch)

In [ ]:

sub.plot_groups_on_average([idxs_maxhipp, idxs_maxhippsig, idxs_hipp,  idxs_hippsig], colors = ['blue', 'yellow',  'red', 'orange'])

In [ ]:
# List of Sentence Rep channels sent to Kumar for hippocampal index
#channels = pd.DataFrame(sub.keys.get('channels'))
#channels.to_csv('sentencerep_channels.csv', index=True, header=True)


In [5]:
# %% Time Sliding decoding for word tokens
out_scores = score({'heat': 1, 'hoot': 2, 'hot': 3, 'hut': 4}, 0.9, 'lda', 5, 20, sub, [idxs_hipp, idxs_hippsig, idxs_maxhippsig, idxs_maxhipp], 
               conds, window_kwargs, scores, shuffle=False)
dict_to_structured_array(out_scores, 'true_score_4idxs_pca90.npy')

# out_scores2 = score({'heat': 1, 'hoot': 2, 'hot': 3, 'hut': 4},
#                                 0.5, 'lda', 5, 250, sub, [idxs_hipp, idxs_hippsig, idxs_maxhippsig, idxs_maxhipp], conds,
#                                 window_kwargs, scores,
#                                 shuffle=True)
# dict_to_structured_array(out_scores2, 'shuffle_score_4idxs_pca50.npy')

[Parallel(n_jobs=30)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done   1 tasks      | elapsed:   15.7s
[Parallel(n_jobs=30)]: Done   2 tasks      | elapsed:   15.8s
[Parallel(n_jobs=30)]: Done   3 tasks      | elapsed:   15.9s
[Parallel(n_jobs=30)]: Done   4 tasks      | elapsed:   15.9s
[Parallel(n_jobs=30)]: Done   5 tasks      | elapsed:   16.0s
[Parallel(n_jobs=30)]: Done   6 tasks      | elapsed:   16.1s
[Parallel(n_jobs=30)]: Done   7 tasks      | elapsed:   16.1s
[Parallel(n_jobs=30)]: Done   8 tasks      | elapsed:   16.1s
[Parallel(n_jobs=30)]: Done   9 tasks      | elapsed:   16.2s
[Parallel(n_jobs=30)]: Done  10 tasks      | elapsed:   16.2s
[Parallel(n_jobs=30)]: Done  11 tasks      | elapsed:   16.2s
[Parallel(n_jobs=30)]: Done  12 tasks      | elapsed:   16.2s
[Parallel(n_jobs=30)]: Done  13 tasks      | elapsed:   16.2s
[Parallel(n_jobs=30)]: Done  14 tasks      | elapsed:   16.3s
[Parallel(n_jobs=30)]: Done  15 tasks      | elapsed:  

Hipp-aud_ls-aud_lm


[Parallel(n_jobs=30)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done   1 tasks      | elapsed:    9.4s
[Parallel(n_jobs=30)]: Done   2 tasks      | elapsed:    9.7s
[Parallel(n_jobs=30)]: Done   3 tasks      | elapsed:    9.7s
[Parallel(n_jobs=30)]: Done   4 tasks      | elapsed:    9.8s
[Parallel(n_jobs=30)]: Done   5 tasks      | elapsed:    9.8s
[Parallel(n_jobs=30)]: Done   6 tasks      | elapsed:    9.9s
[Parallel(n_jobs=30)]: Done   7 tasks      | elapsed:    9.9s
[Parallel(n_jobs=30)]: Done   8 tasks      | elapsed:   10.0s
[Parallel(n_jobs=30)]: Done   9 tasks      | elapsed:   10.0s
[Parallel(n_jobs=30)]: Done  10 tasks      | elapsed:   10.1s
[Parallel(n_jobs=30)]: Done  11 tasks      | elapsed:   10.1s
[Parallel(n_jobs=30)]: Done  12 tasks      | elapsed:   10.1s
[Parallel(n_jobs=30)]: Done  13 tasks      | elapsed:   10.1s
[Parallel(n_jobs=30)]: Done  14 tasks      | elapsed:   10.1s
[Parallel(n_jobs=30)]: Done  15 tasks      | elapsed:  

Hipp-go_ls-go_lm


[Parallel(n_jobs=30)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done   1 tasks      | elapsed:    3.8s
[Parallel(n_jobs=30)]: Done   2 tasks      | elapsed:    3.8s
[Parallel(n_jobs=30)]: Done   3 tasks      | elapsed:    3.8s
[Parallel(n_jobs=30)]: Done   4 tasks      | elapsed:    3.8s
[Parallel(n_jobs=30)]: Done   5 tasks      | elapsed:    3.8s
[Parallel(n_jobs=30)]: Done   6 tasks      | elapsed:    3.8s
[Parallel(n_jobs=30)]: Done   7 tasks      | elapsed:    3.8s
[Parallel(n_jobs=30)]: Done   8 tasks      | elapsed:    3.8s
[Parallel(n_jobs=30)]: Done   9 tasks      | elapsed:    3.8s
[Parallel(n_jobs=30)]: Done  10 tasks      | elapsed:    3.8s
[Parallel(n_jobs=30)]: Done  11 tasks      | elapsed:    3.8s
[Parallel(n_jobs=30)]: Done  12 tasks      | elapsed:    3.8s
[Parallel(n_jobs=30)]: Done  13 tasks      | elapsed:    3.8s
[Parallel(n_jobs=30)]: Done  14 tasks      | elapsed:    3.8s
[Parallel(n_jobs=30)]: Done  15 tasks      | elapsed:  

...

MaxHipp-aud_ls-aud_lm


[Parallel(n_jobs=30)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done   1 tasks      | elapsed:   23.0s
[Parallel(n_jobs=30)]: Done   2 tasks      | elapsed:   23.5s
[Parallel(n_jobs=30)]: Done   3 tasks      | elapsed:   23.5s
[Parallel(n_jobs=30)]: Done   4 tasks      | elapsed:   23.5s
[Parallel(n_jobs=30)]: Done   5 tasks      | elapsed:   23.9s
[Parallel(n_jobs=30)]: Done   6 tasks      | elapsed:   24.0s
[Parallel(n_jobs=30)]: Done   7 tasks      | elapsed:   24.2s
[Parallel(n_jobs=30)]: Done   8 tasks      | elapsed:   24.2s
[Parallel(n_jobs=30)]: Done   9 tasks      | elapsed:   24.2s
[Parallel(n_jobs=30)]: Done  10 tasks      | elapsed:   24.2s
[Parallel(n_jobs=30)]: Done  11 tasks      | elapsed:   24.3s
[Parallel(n_jobs=30)]: Done  12 tasks      | elapsed:   24.3s
[Parallel(n_jobs=30)]: Done  13 tasks      | elapsed:   24.3s
[Parallel(n_jobs=30)]: Done  14 tasks      | elapsed:   24.3s
[Parallel(n_jobs=30)]: Done  15 tasks      | elapsed:  

MaxHipp-go_ls-go_lm


[Parallel(n_jobs=30)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done   1 tasks      | elapsed:    7.4s
[Parallel(n_jobs=30)]: Done   2 tasks      | elapsed:    7.6s
[Parallel(n_jobs=30)]: Done   3 tasks      | elapsed:    7.6s
[Parallel(n_jobs=30)]: Done   4 tasks      | elapsed:    7.6s
[Parallel(n_jobs=30)]: Done   5 tasks      | elapsed:    7.6s
[Parallel(n_jobs=30)]: Done   6 tasks      | elapsed:    7.6s
[Parallel(n_jobs=30)]: Done   7 tasks      | elapsed:    7.6s
[Parallel(n_jobs=30)]: Done   8 tasks      | elapsed:    7.6s
[Parallel(n_jobs=30)]: Done   9 tasks      | elapsed:    7.6s
[Parallel(n_jobs=30)]: Done  10 tasks      | elapsed:    7.6s
[Parallel(n_jobs=30)]: Done  11 tasks      | elapsed:    7.6s
[Parallel(n_jobs=30)]: Done  12 tasks      | elapsed:    7.6s
[Parallel(n_jobs=30)]: Done  13 tasks      | elapsed:    7.7s
[Parallel(n_jobs=30)]: Done  14 tasks      | elapsed:    7.7s
[Parallel(n_jobs=30)]: Done  15 tasks      | elapsed:  

MaxHipp-resp


In [3]:
# %% Plotting
data_dir = os.path.expanduser(os.path.join('~', 'Documents', 'git', 'SentenceRep_analysis', 'analysis', 'decoding'))

# %% Imports for total GroupData
true_scores = np.load(os.path.join(data_dir,'true_score_4idxs_pca90_quadraticDA.npy'), allow_pickle=True)[0]
true_scores = {name: true_scores[name] for name in true_scores.dtype.names}
plots = {}
for key, values in true_scores.items():
    if values is None:
        continue
    plots[key] = np.mean(values.T[np.eye(4).astype(bool)].T, axis=2)
fig, axs = plot_all_scores(plots, conds, {n: i for n, i in zip(names, [0,0,0,0])}, colors, "Word Decoding")

for ax in fig.axes:
    ax.axhline(0.25, color='k', linestyle='--')

# %% Time Sliding decoding significance
shuffle_score = np.load(os.path.join(data_dir, 'shuffle_score_4idxs.npy'), allow_pickle=True)[0]
shuffle_score = {name: shuffle_score[name] for name in shuffle_score.dtype.names}
signif = {}
for cond, score in true_scores.items():
    true = np.mean(score.T[np.eye(4).astype(bool)].T, axis=2)
    shuffle = np.mean(shuffle_score[cond].T[np.eye(4).astype(bool)].T, axis=2)
    signif[cond] = ieeg.calc.stats.time_perm_cluster(true.T, shuffle.T, 0.01, stat_func=lambda x, y, axis: np.mean(x, axis=axis))

#% Plot significance
for cond, ax in zip(conds, axs):
    bars = []
    if isinstance(cond, list):
        cond = "-".join(cond)
    for i, idx in enumerate(idxs):
        name = "-".join([names[i], cond])
        if name.endswith('resp'):
            times = (-1, 1)
        else:
            times = (-0.5, 1.5)
        shuffle = np.mean(shuffle_score[name].T[np.eye(4).astype(bool)].T, axis=2)
        # smooth the shuffle using a window
        window = np.lib.stride_tricks.sliding_window_view(shuffle, 20, 0)
        shuffle = np.mean(window, axis=-1)
        plot_dist_bound(shuffle, 'std', 'both', times, 0, ax=ax, color=colors[i], alpha=0.3)
        bars.append(signif[name])
    #plot_horizontal_bars(ax, bars, 0.05, 'below')
    # Bar width and y positions
    bar_width = 0.01 #LDA run every 10ms intervals with 200ms window 
    y_base_position = 0.5
    height_spacing = 0.02
    if cond in ['aud_ls-aud_lm', 'go_ls-go_lm']:
        xlim = (-0.4, 1.4)
    else:
        xlim = (-0.9, 0.9)
    x_axis = np.append(np.arange(xlim[0], xlim[1], bar_width), xlim[1])
    for idx, (bool_values, p_values) in enumerate(bars):
        y_position = y_base_position + idx * height_spacing  # Shift y position for each item
        # Plot bars for True values in the respective color
        for i, value in enumerate(bool_values):
            if value:
                ax.barh(y=y_position, width=bar_width, height=0.01, left=x_axis[i], color=colors[idx])

fig.show()


NameError: name 'plot_all_scores' is not defined

In [ ]:
from ieeg.viz.ensemble import plot_dist
data = sub.array['zscore','aud_ls'].combine((0, 2))
plot_dist(data.__array__())

In [ ]:
class Person:
    def __init__(self, name, age):
        self.__name = name  # Private attribute
        self.__age = age    # Private attribute

    @property
    def get_name(self):
        return self.__name  # Getter method

    def set_name(self, name):
        self.__name = name  # Setter method

    def get_age(self):
        return self.__age  # Getter method

    def set_age(self, age):
        if age > 0:
            self.__age = age  # Setter method with validation

person = Person("Alice", 30)
print(person.get_age())  # Alice



In [10]:
scores = {'Hipp': None, 'Hipp&Sig': None, 'MaxHipp&Sig': None, 'MaxHipp': None}
names = list(scores.keys())